# Configuration
torch : 1.13.1</br>
cuda : 11.7</br>
torch_geometric : 2.3.0</br>


In [2]:
# %conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 pytorch-cuda=11.7 -c pytorch -c nvidia
# %pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.13.0+cu117.html
# %pip install torch_geometric
# %pip install pandas

     ---------------------------------------- 10.0/10.0 MB 9.4 MB/s eta 0:00:00
     ------------------------------------- 502.3/502.3 kB 30.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch_geometric
print(torch.__version__)
print(torch.version.cuda)
print(torch_geometric.__version__)

c:\Users\PC\.conda\envs\Jupyter_Notebook\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1
11.7
2.3.0


In [38]:
import os
from os.path import join
import torch
from enum import Enum
# from parse import parse_args
import multiprocessing

# os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
# args = parse_args()

# ROOT_PATH = os.path.dirname(os.path.dirname(__file__))
# CODE_PATH = join(ROOT_PATH, 'code')
# DATA_PATH = join(ROOT_PATH, 'data')
# BOARD_PATH = join(CODE_PATH, 'runs')
# FILE_PATH = join(CODE_PATH, 'checkpoints')
# import sys
# sys.path.append(join(CODE_PATH, 'sources'))


# if not os.path.exists(FILE_PATH):
#     os.makedirs(FILE_PATH, exist_ok=True)

config = {}
all_dataset = ['lastfm', 'gowalla', 'yelp2018', 'amazon-book']
all_models  = ['MMGCF']
# config['batch_size'] = 4096
config['emb_size'] = 64
config['bpr_batch_size'] = 1024 # the batch size for bpr loss training procedure"
config['latent_dim_rec'] = 64 # the embedding size
config['n_layers']= 2 # the layer num 
config['dropout'] = 0 # using the dropout or not
config['keep_prob']  = 0.6 # the batch size for bpr loss training procedure
config['A_n_fold'] = 100 # the fold num used to split large adj matrix, like gowalla
config['test_u_batch_size'] = 100 # the batch size of users for testing
config['multicore'] = 0 # whether we use multiprocessing or not in test
config['lr'] = 0.001 # the learning rate
config['decay'] = 1e-4 # the weight decay for l2 normalizaton
config['pretrain'] = 0 # whether we use pretrained weight or not
config['A_split'] = False #
config['bigdata'] = False #

GPU = torch.cuda.is_available()
device = torch.device('cuda' if GPU else "cpu")
CORES = multiprocessing.cpu_count() // 2
seed = 2020

dataset = 'gowalla'
model_name = 'lgn'
if dataset not in all_dataset:
    raise NotImplementedError(f"Haven't supported {dataset} yet!, try {all_dataset}")
if model_name not in all_models:
    raise NotImplementedError(f"Haven't supported {model_name} yet!, try {all_models}")




TRAIN_epochs = 1000
LOAD = 0
PATH = "./checkpoints" # path to save weights
topks = eval("[20]") # @k test list
tensorboard = 1 # enable tensorboard
comment = "lgn"
# let pandas shut up
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)


# Data Loader

In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear

import torch_geometric.transforms as T
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv
from torch_geometric.nn.conv.gcn_conv import gcn_norm

c:\Users\PC\.conda\envs\Jupyter_Notebook\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import pandas as pd

df = pd.read_table('C:\\Users\\PC\\Desktop\\MGCCF\\data\\Gowalla\\filtering_refactored.intr', sep=' ', names=['user','item','time'])
df

,user,item,time
0,0,4169,2010-10-19T23:55:27Z
1,0,31533,2010-10-18T22:17:43Z
2,0,29074,2010-10-17T23:42:03Z
3,0,2572,2010-10-17T19:26:05Z
4,0,40979,2010-10-16T18:50:42Z
...,...,...,...
1027459,29857,13173,2010-04-27T04:32:19Z
1027460,29857,392,2010-03-14T18:13:45Z
1027461,29857,30837,2010-03-13T03:29:55Z
1027462,29857,10578,2010-02-15T03:33:42Z


In [9]:
data = HeteroData()

data['user'].x = torch.tensor(df['user'].unique())
data['item'].x = torch.tensor(df['item'].unique())

data['user', 'interact', 'item'].edge_index = torch.tensor([list(df['user']),list(df['item'])])

In [13]:
class Bipar_gcn():
    

tensor([[    0,     0,     0,  ..., 29857, 29857, 29857],
        [ 4169, 31533, 29074,  ..., 30837, 10578, 24374]])

In [ ]:
class MGE():
    